In [ ]:
import os
import socket
import subprocess
import time
import logging
import msvcrt
import csv

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# ==========================================
# CONFIGURATION
CHROME_DRIVER_PATH = r"C:\Users\ACER\Downloads\chromedriver-win64\chromedriver.exe"
CHROME_PATH = r"C:\Program Files\Google\Chrome\Application\chrome.exe"
USER_DATA_DIR = r"C:\Temp\Gemini_Selenium_Profile" 
# ==========================================

class GeminiAutomation:
    def __init__(self):
        self.driver = None
        self.port = self.find_available_port()
        self.url = "https://aistudio.google.com/prompts/new_chat?model=gemini-2.5-pro"
        
        self.close_existing_chrome()
        self.ensure_profile_dir()
        
        logger.info(f"Launching Chrome on port {self.port}...")
        self.launch_chrome()
        
        if not self.wait_for_port_listening(self.port):
            raise Exception("Chrome failed to start debugging port.")

        logger.info("Attaching WebDriver...")
        self.driver = self.setup_webdriver()
        
        self.wait_for_human_verification()

    def find_available_port(self):
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.bind(('', 0))
            return s.getsockname()[1]

    def close_existing_chrome(self):
        try:
            os.system("taskkill /f /im chrome.exe >nul 2>&1")
            os.system("taskkill /f /im chromedriver.exe >nul 2>&1")
            time.sleep(1)
        except:
            pass

    def ensure_profile_dir(self):
        if not os.path.exists(USER_DATA_DIR):
            os.makedirs(USER_DATA_DIR)

    def launch_chrome(self):
        cmd = [
            CHROME_PATH,
            f"--remote-debugging-port={self.port}",
            f"--user-data-dir={USER_DATA_DIR}",
            "--no-first-run",
            "--no-default-browser-check",
            "--window-size=1200,900",
            self.url
        ]
        subprocess.Popen(cmd)

    def wait_for_port_listening(self, port, timeout=10):
        start = time.time()
        while time.time() - start < timeout:
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
                if s.connect_ex(('127.0.0.1', port)) == 0:
                    return True
            time.sleep(0.5)
        return False

    def setup_webdriver(self):
        options = webdriver.ChromeOptions()
        options.binary_location = CHROME_PATH
        options.add_experimental_option("debuggerAddress", f"127.0.0.1:{self.port}")
        service = Service(executable_path=CHROME_DRIVER_PATH)
        driver = webdriver.Chrome(service=service, options=options)
        return driver

    def wait_for_human_verification(self):
        print("\n" + "!"*60)
        print("BROWSER IS READY.")
        print("1. Log in to Google in the opened window.")
        print("2. Ensure the chat box is visible.")
        print("3. Press ENTER here to start the menu.")
        print("!"*60 + "\n")
        input("Press Enter to continue...")

    def reset_session(self):
        """Refreshes the page to start a completely new chat session"""
        logger.info("Resetting session/Creating new playground...")
        self.driver.get(self.url)
        time.sleep(3) # Wait for page load

    def send_prompt(self, prompt_text):
        if not self.driver: return
        wait = WebDriverWait(self.driver, 15)
        try:
            try:
                input_box = wait.until(EC.element_to_be_clickable((By.TAG_NAME, "textarea")))
            except:
                input_box = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div[contenteditable='true']")))
                
            input_box.click()
            time.sleep(0.5)
            # Clear input box just in case
            input_box.send_keys(Keys.CONTROL, "a")
            input_box.send_keys(Keys.BACKSPACE)
            
            input_box.send_keys(prompt_text)
            time.sleep(0.5)
            input_box.send_keys(Keys.CONTROL, Keys.ENTER)
            
            self.wait_for_response()
            
        except Exception as e:
            logger.error(f"Failed to send prompt: {e}")

    def wait_for_response(self):
        logger.info("Waiting for response... (Press Esc to stop waiting)")
        time.sleep(2) 
        last_text = ""
        stable_count = 0
        
        for _ in range(240): 
            if msvcrt.kbhit() and msvcrt.getch() == b'\x1b':
                logger.info("Esc pressed! Stopping wait.")
                break

            current_text = self.get_latest_response()
            if current_text and current_text == last_text and len(current_text) > 0:
                stable_count += 1
                if stable_count >= 4: 
                    break
            else:
                stable_count = 0
            
            last_text = current_text
            time.sleep(0.5)

    def get_latest_response(self):
        try:
            chunks = self.driver.find_elements(By.CSS_SELECTOR, "ms-text-chunk")
            if not chunks:
                chunks = self.driver.find_elements(By.CSS_SELECTOR, ".markdown-renderer")
            if chunks:
                return chunks[-1].text
        except:
            pass
        return ""

    def quit(self):
        if self.driver:
            self.driver.quit()



# ==========================================
# MAIN LOGIC
# ==========================================

def run_manual_mode(bot):
    print("\n--- MANUAL MODE ---")
    print("Type 'exit' to quit manual mode.")
    
    while True:
        user_input = input("\nEnter Prompt: ")
        if user_input.strip().lower() == 'exit':
            break
        
        bot.send_prompt(user_input)
        response = bot.get_latest_response()
        
        print("\n[Gemini]:")
        print(response)
        print("-" * 40)

def run_file_mode(bot):
    print("\n--- FILE MODE ---")
    print("(Press Esc during processing to stop file mode)")
    file_path = input("Enter the path to your questions txt file (e.g., questions.txt): ").strip()
    
    if not os.path.exists(file_path):
        print("Error: File not found.")
        return

    output_file = "gemini_answers.txt"
    
    with open(file_path, 'r', encoding='utf-8') as f:
        questions = [line.strip() for line in f if line.strip()]

    total = len(questions)
    print(f"Found {total} questions. Processing...")

    for index, question in enumerate(questions, 1):
        if msvcrt.kbhit() and msvcrt.getch() == b'\x1b':
            print("\n[!] Esc pressed. Exiting File Mode...")
            break

        print(f"Processing {index}/{total}: {question[:50]}...")
        
        bot.send_prompt(question)
        
        if msvcrt.kbhit() and msvcrt.getch() == b'\x1b':
            print("\n[!] Esc pressed. Exiting File Mode...")
            break

        response = bot.get_latest_response()
        
        with open(output_file, 'a', encoding='utf-8') as out:
            out.write(f"Question {index}: {question}\n")
            out.write(f"Answer:\n{response}\n")
            out.write("="*60 + "\n\n")
        
        time.sleep(2)

    print(f"\nDone! Answers saved to {output_file}")

def run_csv_pipeline_mode(bot):
    print("\n--- CSV PIPELINE MODE ---")
    print("Logic: 1. Send General Prompt -> 2. Batch CSV Rows -> 3. Save -> 4. Auto-Reset")
    print("(Press Esc during processing to stop)")

    csv_path = input("Enter path to CSV file: ").strip()
    if not os.path.exists(csv_path):
        print("Error: File not found.")
        return
    
    general_prompt = input("Enter the General/System Prompt: ").strip()
    
    try:
        rows_per_prompt = int(input("Enter how many CSV rows to combine per prompt (default 1): ").strip() or "1")
    except ValueError:
        rows_per_prompt = 1
        print("Invalid number, defaulting to 1.")

    try:
        reset_limit = int(input("Enter how many interaction turns before resetting playground (e.g., 5): ").strip() or "5")
    except ValueError:
        reset_limit = 5
        print("Invalid number, defaulting to 5.")

    output_file = "gemini_csv_answers.txt"
    
    rows = []
    try:
        with open(csv_path, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            rows = [" ".join(row) for row in reader if row]
    except Exception as e:
        print(f"Error reading CSV: {e}")
        return

    total_rows = len(rows)
    print(f"Found {total_rows} rows. Processing in batches of {rows_per_prompt}...")

    current_turn_count = 0
    
    for i in range(0, total_rows, rows_per_prompt):
        if msvcrt.kbhit() and msvcrt.getch() == b'\x1b':
            print("\n[!] Esc pressed. Exiting CSV Mode...")
            break

        # Define the batch of data
        batch_rows = rows[i : i + rows_per_prompt]
        # Create a string representation for the prompt (Join with newlines)
        batch_prompt_text = "\n".join(batch_rows)
        
        # Define indexes for logging
        start_idx = i + 1
        end_idx = min(i + rows_per_prompt, total_rows)

        # Check if we need to start a new session (Start of script OR limit reached)
        if current_turn_count == 0:
            print(f"\n[Session] Starting new playground (Reset)...")
            bot.reset_session()
            
            print("[Session] Sending General Prompt...")
            bot.send_prompt(general_prompt)
            # Wait briefly after general prompt (we assume we don't need to save this output)
            time.sleep(1)

        print(f"Processing Rows {start_idx} to {end_idx}...")
        
        bot.send_prompt(batch_prompt_text)

        if msvcrt.kbhit() and msvcrt.getch() == b'\x1b':
            print("\n[!] Esc pressed. Exiting CSV Mode...")
            break

        response = bot.get_latest_response()

        # Save immediately
        with open(output_file, 'a', encoding='utf-8') as out:
            out.write(f"Rows {start_idx}-{end_idx} Input:\n{batch_prompt_text}\n")
            out.write(f"Answer:\n{response}\n")
            out.write("="*60 + "\n\n")

        current_turn_count += 1
        
        # Check if limit reached to reset counter for next iteration
        if current_turn_count >= reset_limit:
            current_turn_count = 0
        
        time.sleep(2)

    print(f"\nDone! Answers saved to {output_file}")

if __name__ == "__main__":
    bot = None
    try:
        bot = GeminiAutomation()
        
        while True:
            print("\nSelect Option:")
            print("1. Manual Input (Type prompts yourself)")
            print("2. File Input (Load from text file)")
            print("3. CSV Pipeline (General Prompt + Batched CSV Data + Auto-Reset)")
            print("4. Exit Script")
            
            choice = input("Choice (1-4): ").strip()
            
            if choice == '1':
                run_manual_mode(bot)
            elif choice == '2':
                run_file_mode(bot)
            elif choice == '3':
                run_csv_pipeline_mode(bot)
            elif choice == '4':
                print("Exiting...")
                break
            else:
                print("Invalid choice.")

    except Exception as e:
        logger.exception("An error occurred:")
    finally:
        if bot:
            bot.quit()

2026-01-11 10:52:45,772 - INFO - Launching Chrome on port 55409...
2026-01-11 10:52:46,794 - INFO - Attaching WebDriver...



!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
BROWSER IS READY.
1. Log in to Google in the opened window.
2. Ensure the chat box is visible.
3. Press ENTER here to start the menu.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


Select Option:
1. Manual Input (Type prompts yourself)
2. File Input (Load from text file)
3. CSV Pipeline (General Prompt + Batched CSV Data + Auto-Reset)
4. Exit Script

--- CSV PIPELINE MODE ---
Logic: 1. Send General Prompt -> 2. Batch CSV Rows -> 3. Save -> 4. Auto-Reset
(Press Esc during processing to stop)


2026-01-11 10:53:23,471 - INFO - Resetting session/Creating new playground...


Found 2385 rows. Processing in batches of 10...

[Session] Starting new playground (Reset)...
[Session] Sending General Prompt...


2026-01-11 10:53:31,214 - INFO - Waiting for response... (Press Esc to stop waiting)


...

Processing Rows 41 to 50...


2026-01-11 11:07:09,905 - ERROR - Failed to send prompt: HTTPConnectionPool(host='localhost', port=52233): Read timed out. (read timeout=120)


Processing Rows 51 to 60...


2026-01-11 11:09:45,424 - ERROR - Failed to send prompt: HTTPConnectionPool(host='localhost', port=52233): Read timed out. (read timeout=120)


Processing Rows 61 to 70...
